### predicción de duración a partir de distancias entre estaciones

Primero intento ver como manejar los datos, es decir, determinar las partes que son importantes. Lo hago con pandas.

Despues de haber hecho eso, ya implementaria las cosas que se tienen que hacer

Digo esto porque, si no está claro cuales son los objetivos y no se consideran futuros problemas, van a surgir complicaciones mas adelante

In [8]:
from math import cos,sqrt,pi

def dist(a, b):
    p = pi / 180
    x = (a[0] * p, a[1] * p)
    y = (b[0] * p, b[1] * p)
    f_m = (x[0] + y[0]) / 2
    d_lat = x[0] - y[0]
    d_lon = x[1] - y[1]
    return sqrt((d_lat)**2 + (cos(f_m) * d_lon)**2)

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
#train = np.genfromtxt('./data/trip_train.csv', delimiter=',')

In [35]:
# duracion
#d[1:,1]

In [29]:
stations = pd.read_csv('./data/station.csv', low_memory=False)

In [7]:
train = pd.read_csv('./data/trip_train.csv', low_memory=False)

In [5]:
l=['start_date','start_station_name','end_station_name','end_date','zip_code','subscription_type','id']
for i in l: 
    try: train.drop(i, axis=1, inplace=True)
    except: pass
#train.head(3)

In [8]:
train.head(3)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107


Ahora voy a intentar ver como separar el set de entrenamiento en set de entrenamiento, validacion y cross validation.


In [16]:
for i in range(1,train.shape[0]/2):
    if train.shape[0]%i==0: print i

1
293
1877


hay 2 numeros que dividen a todos los datos en partes iguales.

podemos tener 293 grupos de 1877 elementos cada uno, o 1877 grupos de 293 elementos.



In [38]:
l=['start_date','start_station_name','end_station_name','end_date','zip_code','subscription_type','installation_date','city','name','dock_count']
for i in l:
    try: stations.drop(i, axis=1, inplace=True)
    except: pass
#stations.head(3)

determino los elementos de tf que se van a utilizar.

en esta parte, planteo como obtener las distancias entre las estaciones, ya que los datos que seran ingresados cuando el algoritmo ya este listo no incluyen esa informacion

Las reglas no dicen nada de entrenar al algoritmo con los datos de weather, stations y status, asi que podemos obtener distancias entre estaciones segun sus coordenadas que estan en stations.

---

sin incluir los sets de datos *extra*, podemos considerar obtener las frecuencias de trayectos. A lo que me refiero es que, hay trayectos que aparecen mas que otros. Con eso en mente podemos relacionar trayectos con un determinado tiempo de duracion, y si el dato a evaluar tiene ese trayecto podemos hacer un estimativo del tiempo de duracion.

tener en cuenta boostrapping y bagging y k-fold


In [29]:
# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)

# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = d[1:,4]
y_train = d[1:,1]#[0,-1,-2,-3]

# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(100):
    sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

V: [ nan] W: [ nan] b: [ nan] loss: nan
